In [ ]:
import sqil_core as sqil
from sqil_core.utils import * 
import matplotlib.pyplot as plt
import numpy as np

# Use `qt` for interactive plots and `inline` for notebook integrated plots
%matplotlib qt

### Basics

#### Reading data

- In `sqil-experiments` data is always assumed to be multi-qubit data, so it's always prefixed by a qubit ID, e.g. `"q0"`. 
- The fields available in data files are always specified in the experiment's `db_schema`

Read data as a dictionary.

In [ ]:
path = r"" # Path to the folder containing the data
datadict = sqil.extract_h5_data(path)

Read data and metadata as a dictionary.  
`metadata` contains the database schema and other information used by analysis scripts.

In [ ]:
path = r"" # Path to the folder containing the data
datadict = sqil.extract_h5_data(path, get_metadata=True)

Read only a specific field.  
Assume you have a qubit spectroscopy. In the `db_schema` of `qu_spec.py` there are two fields: `data` and `frequencies`. If you only need the `data` field for qubit `q0` it can be extracted as follows


In [ ]:
path = r"" # Path to the folder containing the data
data = sqil.extract_h5_data(path, ["q0/data"])

### Re-plotting experiments

Can be useful to plot old data and make it interactive (using `%matplotlib qt`).  
The general idea is to import the analysis function directly from the experimental file and pass to it the data.  

For complex analysis that requires knowledge of other qubit parameters, it may be necessary to pass the qpu file along with the data.  
It is recommended to use `qpu_old.json`, as it contains the qubit paramters before running the experiment.

#### Readout resonator spectroscopy

In [ ]:
from sqil_experiments.measurements.rr_spec import rr_spec_analysis

path = r""

datadict = sqil.extract_h5_data(path, get_metadata=True)
qpu = sqil.read_qpu(path, "qpu_old.json")

rr_spec_analysis(datadict=datadict, qpu=qpu)
plt.show()

#### Qubit spectroscopy

In [ ]:
from sqil_experiments.measurements.qu_spec import qu_spec_analysis

path = r""

datadict = sqil.extract_h5_data(path, get_metadata=True)
qpu = sqil.read_qpu(path, "qpu_old.json")

qu_spec_analysis(datadict=datadict, qpu=qpu)
plt.show()

#### T1

In [ ]:
from sqil_experiments.measurements.T1 import analyze_T1

path = r""

datadict = sqil.extract_h5_data(path, get_metadata=True)
qpu = sqil.read_qpu(path, "qpu_old.json")

analyze_T1(datadict=datadict, qpu=qpu)
plt.show()

#### T2 echo

In [ ]:
from sqil_experiments.measurements.T2_echo import analyze_T2_echo

path = r""

datadict = sqil.extract_h5_data(path, get_metadata=True)
qpu = sqil.read_qpu(path, "qpu_old.json")

analyze_T2_echo(datadict=datadict, qpu=qpu)
plt.show()

### Custom analysis

#### IQ Plane

Verify that Rabi, T1 and T2 are on the same line in the IQ plane

In [ ]:
%matplotlib inline
fig, ax = plt.subplots(1,1, figsize=(5,5))

path_blobs = r"Z:\Projects\Stormcrow\data\20251110_stormcrow_N17\2025-12-02\01120-single_shot_hdawg_2025-12-02T125717"
g,e = extract_h5_data(path_blobs, ["q0/g", "q0/e"])
ax.scatter(np.mean(g.real), np.mean(g.imag), marker="v", color="cyan", label=r"$|g\rangle$", zorder=100)
ax.scatter(np.mean(e.real), np.mean(e.imag), marker="^", color="magenta", label=r"$|e\rangle$", zorder=100)

path_rabi = r"Z:\Projects\Stormcrow\data\20251110_stormcrow_N17\2025-12-02\01113-time_rabi_2025-12-02T123213"
data_rabi = extract_h5_data(path_rabi, ["q0/data"])
ax.plot(data_rabi.real, data_rabi.imag, 'o', label="Rabi")

path_T1 = r"Z:\Projects\Stormcrow\data\20251110_stormcrow_N17\2025-12-02\01152-T1_hdawg_2025-12-02T141155"
data_T1 = extract_h5_data(path_T1, ["q0/data"])
ax.plot(data_T1.real, data_T1.imag, 'o', label="T1")

path_T2 = r"Z:\Projects\Stormcrow\data\20251110_stormcrow_N17\2025-12-02\01163-T2_echo_hdawg_2025-12-02T143504"
data_T2 = extract_h5_data(path_T2, ["q0/data"])
ax.plot(data_T2.real, data_T2.imag, 'o', label="T2")

ax.axis('equal')
ax.grid(True)
ax.legend()
plt.show()